In [3]:
# imports
import torch
import utils
import my_models
import hyperparameters as h # this prints GPU enabled = True


files = ['sample-music.txt', 'input.txt']
# load the inputs as a list of ints
inputs, char2int_cypher, int2char_cypher = utils.load_music(files[1], use_custom=True)
# full input.txt is 501470 in length
dict_size = len(char2int_cypher) # conversion is the dict convert char to int

h.char2int_cypher = char2int_cypher
h.int2char_cypher = int2char_cypher


# define test and validation set
split = int(len(inputs) * 0.1) # change 0.1 to how big we want validation set to be
validation_set = inputs[:split]
training_set = inputs[split:]


# create model
lstm = my_models.lstm_char_rnn(dict_size, h.hidden_size, h.num_hidden_layers, batch_size=h.batch_size)
init_hidden = lstm.initialize_hidden()
if h.GPU:
    init_hidden = init_hidden.cuda()
    lstm.cuda()

optimizer_lstm = torch.optim.Adam(lstm.parameters(), lr=0.01)

Using custom starts and ends...
There are 95 unique characters in this dataset



In [6]:


my_models.generate(h.save_file, lstm, h.temperature, h.prediction_length)
print()
my_models.generate(h.save_file_progress, lstm, h.temperature, h.prediction_length)

final output =  $
X:51
T:Jich
W: I no K\'an hn-hornp-pea-5 dac Hse (16738))
O:France
D:Jack Drion 
M:C|
K:Em
~A3 efg|fed efe|dcB cBc|Ace fed|efg afg|
afdc d^cd|efe edB | cde add|gBe afe g2g|ag~f2 ABAG||:d^ca aGAG|
D2GB A2de:|2 ~e3d ede^f|1 gedc BAG:|
|:ec ef/e/|fB AF/A/|dA Ac|ef gf|ef ed|c2 c/B/A|FGBc|1 d4 :| [4E42C | B2>G2 | B4B2c/4B>c/2d/2 | eB z/2B/2B/2/2/2-5
M:C|
K:Gmixlimhos jig
H:Adowan
Z:id:hn-hornpipe-42
M:C
R:Dice tonnapate
R:march
O:France
A:Provence
C:
A2 ABd|c2dB AGAF|G2AB c2ed|BAGE G2gf||
g2 gecA FG

final output =  $
X:82
T:Sellele@gies to Boithau
G:Ldarchou
C:vaoiling to | fd | cB AF FB | BB AB | A3) AG Ad | fd e2 |
_B2 BG GB | B4 ::
G>c dc | B2 B2 | B4- | A4 |
c>c ef | g4 | f>e d2cc | B>A G3) | (3B>B BA | Bc d/d/c/B/ | c/B/ AB |1 AB cA :|
P:variations:
|:ag ag/e/|de/d/ g3/a/2g/2|a3/2g/2e2d2c/2d/2|e/2f/2 gf ^g|
a>g | +breath+b2+breath+F2)D | E2G GFG | A2F AFE |
EF/F/ GEF |EF/E/ EG|FA B/c/B/A/|Bc df|ea e/c/B/A/|df A2:|
P:Variations
A2 BA|eA ce|df fe|1 dB B>G|AA B/c/B/A/G/

1